In [ ]:
pip install transformers

     |████████████████████████████████| 2.9 MB 15.0 MB/s 
     |████████████████████████████████| 895 kB 65.7 MB/s 
     |████████████████████████████████| 3.3 MB 75.4 MB/s 
     |████████████████████████████████| 636 kB 76.7 MB/s 
     |████████████████████████████████| 52 kB 1.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import torch
import torch.nn as nn

import transformers
from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

df = pd.read_csv('drive/MyDrive/MSR_data_0910/0906.csv')
df

,Unnamed: 0,Text,Value,label
0,0,10/19/2020\nSLAVERY AND HUMAN TRAFFICKING\nINT...,"Whistleblower protection (direct employees), W...",1
1,1,Pension\nProtection\nFund\nPension\nProtection...,"Whistleblower protection (direct employees), I...",0
2,2,10/19/2020\nModern Slavery & Human\nTroffickin...,Whistleblower protection (direct employees),1
3,3,Greene King I MODERN SLAVERY STATEMENT\nMODERN...,Whistleblower protection (direct employees),1
4,4,Charles Wells Ltd\nModern Slavery and Human Tr...,Whistleblower protection (direct employees),1
...,...,...,...,...
1376,1376,Modern Slavery Act 2015 Statement\nOur Policy\...,No,0
1377,1377,Upto 60% off + 20% off ends\n\nSearch for\npro...,No,0
1378,1378,Slavery and Human Trafficking Transparency Sta...,Whistleblower protection (direct employees),1
1379,1379,Reward Gateway Slavery\nand Human Trafficking\...,"Hotline (direct employees), Whistleblower prot...",1


In [ ]:
df.label.value_counts()

1    735
0    646
Name: label, dtype: int64

In [ ]:
data_tensor = torch.load('drive/MyDrive/MSR_data_0910/data_tensor_0925')

print(data_tensor.size())

torch.Size([1381, 200, 768])


In [ ]:
aug_tensor = torch.load('drive/MyDrive/MSR_data_0910/aug_tensor_0922')
print(aug_tensor.size())

aug_label = torch.load('drive/MyDrive/MSR_data_0910/aug_label')
print(aug_label.size())

I_tensor_aug = torch.load('drive/MyDrive/MSR_data_0910/I_tensor_aug')
print(I_tensor_aug.size())

torch.Size([254, 200, 768])
torch.Size([254])
torch.Size([254, 1, 200])


#### I Tensor

In [ ]:
I_tensor = torch.load('drive/MyDrive/MSR_data_0910/I_tensor_0925')

print(I_tensor.size())


torch.Size([1381, 1, 200])


### Model training

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
label = df['label'].tolist()
len(label)

1381

In [ ]:
label = df['label'].tolist()

train_x = data_tensor[:1000]
#train_x = torch.cat((train_x, aug_tensor[150:]), 0)
print(train_x.shape)
train_x = train_x.permute(0,2,1)


train_y = torch.tensor(label[:1000])
#train_y = torch.cat((train_y, aug_label[150:]), 0)
print(train_y.shape)

train_i = I_tensor[:1000]
#train_i = torch.cat((train_i, I_tensor_aug[150:]), 0)
print(train_i.shape)

val_x = data_tensor[1000:]
val_x = val_x.permute(0,2,1)
val_y = torch.tensor(label[1000:])
val_i = I_tensor[1000:]

test_x = data_tensor[1000:]
test_x = test_x.permute(0,2,1)
test_y = torch.tensor(label[1000:])
test_i = I_tensor[1000:]


torch.Size([1000, 200, 768])
torch.Size([1000])
torch.Size([1000, 1, 200])


In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 64

train_data = TensorDataset(train_x, train_y, train_i)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler = train_sampler, batch_size=batch_size, shuffle=False)

val_data = TensorDataset(val_x, val_y, val_i)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size, shuffle=False)

In [ ]:
import torch
a2 = torch.FloatTensor([0.3, 0.5, 0.2])
a1 = torch.FloatTensor([0.0000000001, 0.0000000001, 0.9999999998])
print(a1.sum())
c1 = torch.nn.KLDivLoss(size_average=False)(a1.log(),a2)
print(c1)

tensor(1.)
tensor(17.3910)


/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [ ]:
class model(nn.Module):
    def __init__(self):
        super(model, self).__init__()        
        self.conv1 = nn.Conv1d(in_channels = 768, out_channels=1, kernel_size=1, stride=1, padding=0)
        self.conv2 = nn.Conv1d(in_channels = 768, out_channels=100, kernel_size=1, stride=1, padding=0)

        self.fc1 = nn.Linear(100, 100)
        self.fc2 = nn.Linear(100, 2)  

        self.softmax1 = nn.Softmax(dim=-1)
        self.relu =  nn.ReLU()

        self.conv1_bn = nn.BatchNorm1d(1)
        self.conv2_bn = nn.BatchNorm1d(100)
        self.dense1_bn = nn.BatchNorm1d(100)

        self.dropout = nn.Dropout(0.1)
      
    def forward(self, x, y, i):        
        
        alpha = self.conv1_bn(self.conv1(x)) 

        alpha = alpha + lbda * i  

        alpha_prime = self.softmax1(alpha) 
        alpha_prime_permute = alpha_prime.permute(0,2,1)    

        x_2 = self.relu(self.conv2_bn(self.conv2(x)))                        
        h = torch.matmul(x_2, alpha_prime_permute)         
        h = h.view(h.shape[0], -1)           
        
        output = self.relu(self.dense1_bn(self.fc1(h))) 
        output = self.dropout(output)  
        output = self.fc2(output)  

        alpha_prime1 = alpha_prime.view(alpha_prime.shape[0], -1)   

        return alpha_prime1, output


In [ ]:
def train():
    Model.train()
    total_loss = 0
    train_correct = 0
    train_total = 0
    
    for step, batch in enumerate(train_dataloader):

        batch = [r.to(device) for r in batch]
        x, y, i = batch
        Model.zero_grad()        
        
        alpha_pred, output_preds = Model(x, y, i)

        i = i.view(i.shape[0], -1)
        softmax1 = nn.Softmax(dim=-1)
        i = softmax1(i)

        alpha_log = alpha_pred.log()
        
        loss = beta*nn.KLDivLoss(size_average=False)(alpha_pred.log(),i) + cross_entropy(output_preds, y)
        total_loss = total_loss + loss.item()

        #print('KLloss', beta*nn.KLDivLoss(size_average=False)(alpha_pred.log(),i), 'CEloss', cross_entropy(output_preds, y))

        loss.backward()
        optimizer.step()
        
        _, predicted = torch.max(output_preds.data, 1)
        train_total += y.size(0)
        train_correct += (predicted == y).sum().item()
        output_preds = output_preds.detach().cpu().numpy()   
        
    avg_loss = total_loss / len(train_dataloader) 
    train_acc = train_correct/train_total
  
    return avg_loss, train_acc 


In [ ]:
def evaluate():
    
    print("\nEvaluating...")
    Model.eval()   # deactivate dropout layers
    total_loss = 0
    val_total = 0
    val_correct = 0

    for step,batch in enumerate(val_dataloader):   
        batch = [t.to(device) for t in batch] # push the batch to gpu
        x, y, i = batch
        
        with torch.no_grad(): # deactivate autograd 
            alpha_pred, output_preds = Model(x, y, i)

            i = i.view(i.shape[0], -1)
            softmax1 = nn.Softmax(dim=-1)
            i = softmax1(i)

            #print(alpha_pred.log())
            loss = beta*nn.KLDivLoss(size_average=False)(alpha_pred.log(),i) + cross_entropy(output_preds, y)
            
            print('KLloss', beta*nn.KLDivLoss(size_average=False)(alpha_pred.log(),i), 'CEloss', cross_entropy(output_preds, y))

            total_loss = total_loss + loss.item()

            _, predicted = torch.max(output_preds.data, 1)
            val_total += y.size(0)
            val_correct += (predicted == y).sum().item()
            output_preds = output_preds.detach().cpu().numpy()
            
    avg_loss = total_loss / len(val_dataloader) 
    val_acc = val_correct/val_total

    return avg_loss, val_acc

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
 
class_wts = compute_class_weight('balanced', classes= torch.unique(train_y).tolist(), y=train_y.tolist())
print(class_wts)
weights = torch.tensor(class_wts, dtype = torch.float)
weights = weights.to(device)

from torch import optim
Model = model()
Model = Model.to(device)
cross_entropy = nn.CrossEntropyLoss(weight=weights)
MSEloss = nn.MSELoss()
KLloss = nn.KLDivLoss()

from transformers import AdamW

epochs = 100

[1.01010101 0.99009901]


In [ ]:
import numpy as np

best_valid_acc = 0

train_losses=[]
valid_losses=[]
train_acces = []
val_acces = []

for epoch in range(epochs):

  if epoch <= 10:
    beta = 0.1
    lbda = 0
    lr = 0.001
    
    
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs), 'beta =', beta, 'lbda =', lbda, 'lr', lr)

    optimizer = optim.AdamW(Model.parameters(), lr = lr, weight_decay=0.01)

    train_loss, train_acc = train() 
    valid_loss, val_acc = evaluate() 

    if val_acc > best_valid_acc: 
        best_valid_acc = val_acc
        torch.save(Model.state_dict(), '/content/drive/MyDrive/MSR_data_0910/kl_2stages_weights0929.pt')
    
    train_acces.append(train_acc)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    val_acces.append(val_acces)
    
    print('Train acc', np.round(train_acc, 3), 'Val acc', np.round(val_acc, 3))
    print('Train loss', np.round(train_loss, 3), 'Val loss', np.round(valid_loss, 3))

  else:
    beta = 0
    lmda = 5
    lr = 0.001
    
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs), 'beta =', beta, 'lbda =', lmda, 'lr', lr)

    optimizer = optim.AdamW(Model.parameters(), lr = lr, weight_decay=0.01)

    train_loss, train_acc = train() 
    valid_loss, val_acc = evaluate() 

    if val_acc > best_valid_acc: 
        best_valid_acc = val_acc
        torch.save(Model.state_dict(), '/content/drive/MyDrive/MSR_data_0910/kl_2stages_weights0929.pt')
    
    train_acces.append(train_acc)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    val_acces.append(val_acces)
    
    print('Train acc', np.round(train_acc, 3), 'Val acc', np.round(val_acc, 3))
    print('Train loss', np.round(train_loss, 3), 'Val loss', np.round(valid_loss, 3))




 Epoch 1 / 100 beta = 0.1 lbda = 0 lr 0.001


/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Evaluating...
KLloss tensor(1.1184, device='cuda:0') CEloss tensor(0.6725, device='cuda:0')
KLloss tensor(0.9972, device='cuda:0') CEloss tensor(0.6948, device='cuda:0')
KLloss tensor(1.3071, device='cuda:0') CEloss tensor(0.6869, device='cuda:0')
KLloss tensor(1.3515, device='cuda:0') CEloss tensor(0.6858, device='cuda:0')
KLloss tensor(1.2574, device='cuda:0') CEloss tensor(0.6937, device='cuda:0')
KLloss tensor(0.9833, device='cuda:0') CEloss tensor(0.6924, device='cuda:0')
Train acc 0.571 Val acc 0.507
Train loss 2.419 Val loss 1.857

 Epoch 2 / 100 beta = 0.1 lbda = 0 lr 0.001

Evaluating...
KLloss tensor(2.6203, device='cuda:0') CEloss tensor(0.6873, device='cuda:0')
KLloss tensor(2.2668, device='cuda:0') CEloss tensor(0.6807, device='cuda:0')
KLloss tensor(3.0801, device='cuda:0') CEloss tensor(0.6817, device='cuda:0')
KLloss tensor(3.1697, device='cuda:0') CEloss tensor(0.6964, device='cuda:0')
KLloss tensor(2.9066, device='cuda:0') CEloss tensor(0.6820, device='cuda:0')
KLlos

In [ ]:
path = '/content/drive/MyDrive/MSR_data_0910/kl_2stages_weights0929.pt'
Model.load_state_dict(torch.load(path))

for param_tensor in Model.state_dict():
    print(param_tensor, "\t", Model.state_dict()[param_tensor].size())

conv1.weight 	 torch.Size([1, 768, 1])
conv1.bias 	 torch.Size([1])
conv2.weight 	 torch.Size([100, 768, 1])
conv2.bias 	 torch.Size([100])
fc1.weight 	 torch.Size([100, 100])
fc1.bias 	 torch.Size([100])
fc2.weight 	 torch.Size([2, 100])
fc2.bias 	 torch.Size([2])
conv1_bn.weight 	 torch.Size([1])
conv1_bn.bias 	 torch.Size([1])
conv1_bn.running_mean 	 torch.Size([1])
conv1_bn.running_var 	 torch.Size([1])
conv1_bn.num_batches_tracked 	 torch.Size([])
conv2_bn.weight 	 torch.Size([100])
conv2_bn.bias 	 torch.Size([100])
conv2_bn.running_mean 	 torch.Size([100])
conv2_bn.running_var 	 torch.Size([100])
conv2_bn.num_batches_tracked 	 torch.Size([])
dense1_bn.weight 	 torch.Size([100])
dense1_bn.bias 	 torch.Size([100])
dense1_bn.running_mean 	 torch.Size([100])
dense1_bn.running_var 	 torch.Size([100])
dense1_bn.num_batches_tracked 	 torch.Size([])


In [ ]:
path = '/content/drive/MyDrive/MSR_data_0910/kl_2stages_weights0929.pt'
Model.load_state_dict(torch.load(path))

print('Train_report:')
with torch.no_grad():
  _, train_preds = Model(train_x.to(device), train_y.to(device), train_i.to(device))

  train_preds = train_preds.detach().cpu().numpy()

train_preds = np.argmax(train_preds, axis = 1)
print(classification_report(train_y, train_preds))

print('Val_report:')
with torch.no_grad():
  _, val_preds = Model(val_x.to(device), val_y.to(device), val_i.to(device))
  val_preds = val_preds.detach().cpu().numpy()

val_preds = np.argmax(val_preds, axis = 1)
print(classification_report(val_y, val_preds))

print('Test_report:')
with torch.no_grad():
  _, test_preds = Model(test_x.to(device), test_y.to(device), test_i.to(device))
  test_preds = test_preds.detach().cpu().numpy()

test_preds = np.argmax(test_preds, axis = 1)
print(classification_report(test_y, test_preds))

Train_report:
              precision    recall  f1-score   support

           0       0.85      0.64      0.73       495
           1       0.72      0.89      0.79       505

    accuracy                           0.76      1000
   macro avg       0.78      0.76      0.76      1000
weighted avg       0.78      0.76      0.76      1000

Val_report:
              precision    recall  f1-score   support

           0       0.71      0.43      0.54       151
           1       0.70      0.89      0.78       230

    accuracy                           0.71       381
   macro avg       0.71      0.66      0.66       381
weighted avg       0.71      0.71      0.69       381

Test_report:
              precision    recall  f1-score   support

           0       0.71      0.43      0.54       151
           1       0.70      0.89      0.78       230

    accuracy                           0.71       381
   macro avg       0.71      0.66      0.66       381
weighted avg       0.71      0.71  

In [ ]:
val_y_list = val_y.tolist()
val_preds_list  = val_preds.tolist()

ind_list = list(range(1000, 1381))

incorrect_pred = []
val_FN = []
val_FP = []
for i, a, b in zip(ind_list, val_y_list, val_preds_list):
  if a != b:
    incorrect_pred.append(i)

  if (a == 0 and b == 1):
    val_FP.append(i)
  
  if (a == 1 and b == 0):
    val_FN.append(i)

print(len(incorrect_pred))
print(len(val_FP))
print(len(val_FN))

112
86
26


In [ ]:
train_y_list = train_y.tolist()
train_preds_list  = train_preds.tolist()

ind_list = list(range(0, 1000))

incorrect_pred_train = []
train_FN = []
train_FP = []
for i, a, b in zip(ind_list, train_y_list, train_preds_list):
  if a != b:
    incorrect_pred_train.append(i)

  if (a == 0 and b == 1):
    train_FP.append(i)
  
  if (a == 1 and b == 0):
    train_FN.append(i)

print(len(incorrect_pred_train))
print(len(train_FP))
print(len(train_FN))

236
178
58


In [ ]:
I_tensor_np = I_tensor.cpu().detach().numpy()

keyw_doc_list = []
for i in range(len(I_tensor_np)):
  if 1 in I_tensor_np[i]:
    keyw_doc_list.append(1)
  else:
    keyw_doc_list.append(0)

print(len(keyw_doc_list))

1381


In [ ]:
keyword_pd = pd.Series(keyw_doc_list)
df['keyword'] = keyword_pd 
df

,Unnamed: 0,Text,Value,label,keyword
0,0,10/19/2020\nSLAVERY AND HUMAN TRAFFICKING\nINT...,"Whistleblower protection (direct employees), W...",1,1
1,1,Pension\nProtection\nFund\nPension\nProtection...,"Whistleblower protection (direct employees), I...",0,1
2,2,10/19/2020\nModern Slavery & Human\nTroffickin...,Whistleblower protection (direct employees),1,1
3,3,Greene King I MODERN SLAVERY STATEMENT\nMODERN...,Whistleblower protection (direct employees),1,1
4,4,Charles Wells Ltd\nModern Slavery and Human Tr...,Whistleblower protection (direct employees),1,1
...,...,...,...,...,...
1376,1376,Modern Slavery Act 2015 Statement\nOur Policy\...,No,0,1
1377,1377,Upto 60% off + 20% off ends\n\nSearch for\npro...,No,0,1
1378,1378,Slavery and Human Trafficking Transparency Sta...,Whistleblower protection (direct employees),1,1
1379,1379,Reward Gateway Slavery\nand Human Trafficking\...,"Hotline (direct employees), Whistleblower prot...",1,1


In [ ]:
df['keyword'].value_counts()

1    1017
0     364
Name: keyword, dtype: int64

In [ ]:
key1_lb1_train = []
key1_lb0_train = []
key0_lb1_train = []
key0_lb0_train = []

for i in list(range(0, 1000)):
  if df['keyword'][i] ==1 and df['label'][i] ==1:
    key1_lb1_train.append(i)

  if df['keyword'][i] ==1 and df['label'][i] ==0:
    key1_lb0_train.append(i)

  if df['keyword'][i] ==0 and df['label'][i] ==1:
    key0_lb1_train.append(i)

  if df['keyword'][i] ==0 and df['label'][i] ==0:
    key0_lb0_train.append(i)

print(len(key1_lb1_train))
print(len(key1_lb0_train))
print(len(key0_lb1_train))
print(len(key0_lb0_train))

422
259
83
236


In [ ]:
key1_lb1_val = []
key1_lb0_val = []
key0_lb1_val = []
key0_lb0_val = []

for i in list(range(1000, 1381)):
  if df['keyword'][i] ==1 and df['label'][i] ==1:
    key1_lb1_val.append(i)

  if df['keyword'][i] ==1 and df['label'][i] ==0:
    key1_lb0_val.append(i)

  if df['keyword'][i] ==0 and df['label'][i] ==1:
    key0_lb1_val.append(i)

  if df['keyword'][i] ==0 and df['label'][i] ==0:
    key0_lb0_val.append(i)

print(len(key1_lb1_val))
print(len(key1_lb0_val))
print(len(key0_lb1_val))
print(len(key0_lb0_val))

219
117
11
34


In [ ]:
df_val_FP = df.iloc[val_FP]

print(len(df_val_FP))

count = 0
for i in val_FP:
  if df_val_FP['keyword'][i] == 1:
    count += 1

print(count)

df_val_FP

86
73


,Unnamed: 0,Text,Value,label,keyword
1012,1012,Menu\n\nF\n\nPrivilege\n\n/\n\n$\n\nAdvice\n\n...,No,0,1
1014,1014,2/25/2021\n\nModern Slavery Statement | Hobbyc...,No,0,1
1017,1017,United Kingdom\n\nAnti-Slavery Statement 2020\...,No,0,1
1023,1023,\nSearch for...\n\n\n\n> Modern\nHome Slaver...,No,0,1
1049,1049,Standard Advisory London Limited\n1.\n\nFinanc...,In Development (direct employees),0,1
...,...,...,...,...,...
1360,1360,Contact Us\n\n+1-502-775-7777\n\nCALIFORNIA SU...,No,0,1
1362,1362,Manage my booking\nLatest Travel Advice\n\nHol...,No,0,1
1374,1374,MODERN SLAVERY ACT TRANSPARENCY STATEMENT\nInt...,No,0,1
1377,1377,Upto 60% off + 20% off ends\n\nSearch for\npro...,No,0,1


In [ ]:
df_val_FN = df.iloc[val_FN]

print(len(df_val_FN))

count = 0
for i in val_FN:
  if df_val_FN['keyword'][i] == 1:
    count += 1

print(count)

df_val_FN

26
20


,Unnamed: 0,Text,Value,label,keyword
1008,1008,APPE\n\nMODERN SLAVERY STATEMENT\nThis stateme...,Hotline (direct employees),1,1
1030,1030,About\n\nIntercity Express Programme\n\nGWML\n...,Focal Point (direct employees),1,0
1032,1032,SLAVERY AND HUMAN TRAFFICKING STATEMENT\nThis ...,"Whistleblower protection (direct employees), F...",1,1
1040,1040,"Covid 19, Information & Updates\n\n\n\n\n H...",Hotline (direct employees),1,1
1054,1054,2/25/2021\n\nAnti-Slavery Policy - Ballyvesey ...,"Whistleblower protection (direct employees), F...",1,1
1064,1064,Carclo PLC: Policy on Modern Slavery and Child...,Focal Point (direct employees),1,0
1118,1118,voestalpine High Performance Metals UK\n\nAnti...,Whistleblower protection (direct employees),1,1
1133,1133,CALIFORNIA TRANSPARENCY IN SUPPLY CHAIN\nACT D...,Focal Point (direct employees),1,0
1137,1137,Modern Slavery and Human Trafficking Statement...,Whistleblower protection (direct employees),1,1
1152,1152,\n\nModern Slavery and Human Trafficking Tran...,Whistleblower protection (direct employees),1,1
